<a href="https://colab.research.google.com/github/sn715/SOLA/blob/main/SOLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title IMPORTS
from music21 import *

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from collections import Counter
from tqdm import tqdm
import os
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title READ MIDI
def read_midi(file):

    print("Loading Music File:",file)

    notes=[]
    notes_to_parse = None

    #parsing a midi file
    midi = converter.parse(file)

    #group based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #loop over all the instruments
    for part in s2.parts:

        notes_to_parse = part.recurse()

        #find whether a particular element is note or a chord
        for element in notes_to_parse:

            #note
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))

            #chord
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [ ]:
#@title CLASSICAL CALM DATA

def createClassicalCalmData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    calm_data_list = []

    for x in final_data.index:
        if final_data['calmness'][x] == 1.0:
            calm_data_list.append(x)

    calm_data = pd.DataFrame(calm_data_list)
    calm_data.to_csv('final_calm_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    calmFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in calm_data_list:
            print(i)
            calmFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in calmFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))

    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((26, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((26, 32))

    return bigX, bigY, calm_data, new_music


In [ ]:
#@title CLASSICAL POWER DATA

def createClassicalPowerData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    power_data_list = []

    for x in final_data.index:
        if final_data['power'][x] == 1.0:
            power_data_list.append(x)

    power_data = pd.DataFrame(power_data_list)
    power_data.to_csv('final_power_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    powerFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in power_data_list:
            print(i)
            powerFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in powerFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((4, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((4, 32))

    return bigX, bigY, power_data, new_music


In [ ]:
#@title ROCK CALM DATA

def createRockCalmData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    calm_data_list = []

    for x in final_data.index:
        if final_data['calmness'][x] == 1.0:
            calm_data_list.append(x)

    calm_data = pd.DataFrame(calm_data_list)
    calm_data.to_csv('final_calm_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    calmFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in calm_data_list:
            print(i)
            calmFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in calmFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((12, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((12, 32))

    return bigX, bigY, calm_data, new_music


In [ ]:
#@title ROCK POWER DATA

def createRockPowerData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    power_data_list = []

    for x in final_data.index:
        if final_data['power'][x] == 1.0:
            power_data_list.append(x)

    power_data = pd.DataFrame(power_data_list)
    power_data.to_csv('final_power_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    powerFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in power_data_list:
            print(i)
            powerFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in powerFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((6, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((6, 32))

    return bigX, bigY, power_data, new_music


In [ ]:
#@title ELECTRONIC CALM DATA

def createElectronicCalmData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    calm_data_list = []

    for x in final_data.index:
        if final_data['calmness'][x] == 1.0:
            calm_data_list.append(x)

    calm_data = pd.DataFrame(calm_data_list)
    calm_data.to_csv('final_calm_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    calmFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in calm_data_list:
            print(i)
            calmFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in calmFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((22, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((22, 32))

    return bigX, bigY, calm_data, new_music


In [ ]:
#@title ELECTRONIC POWER DATA

def createElectronicPowerData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    power_data_list = []

    for x in final_data.index:
        if final_data['power'][x] == 1.0:
            power_data_list.append(x)

    power_data = pd.DataFrame(power_data_list)
    power_data.to_csv('final_power_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    powerFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in power_data_list:
            print(i)
            powerFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in powerFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((9, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((9, 32))

    return bigX, bigY, power_data, new_music


In [ ]:
#@title POP CALM DATA

def createPopCalmData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    calm_data_list = []

    for x in final_data.index:
        if final_data['calmness'][x] == 1.0:
            calm_data_list.append(x)

    calm_data = pd.DataFrame(calm_data_list)
    calm_data.to_csv('final_calm_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    calmFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in calm_data_list:
            print(i)
            calmFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in calmFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((19, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((19, 32))

    return bigX, bigY, calm_data, new_music


In [ ]:
#@title POP POWER DATA

def createPopPowerData(musicPath, emotionData):

    #CSV EMOTION DATA

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    power_data_list = []

    for x in final_data.index:
        if final_data['power'][x] == 1.0:
            power_data_list.append(x)

    power_data = pd.DataFrame(power_data_list)
    power_data.to_csv('final_power_data_for_annotation.csv')

    #MUSIC DATA

    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    powerFiles = []

    for i in files:
        if i[0:2] != ".i":
          if int(i[0:2]) in power_data_list:
            print(i)
            powerFiles.append(i)

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in powerFiles])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)
        new_music.append(temp)

    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(26):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((4, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((4, 32))

    return bigX, bigY, power_data, new_music


In [ ]:
#@title BUILD MODEL
def build_seamise_model_with_functional():

    # instantiate the input Tensor
    input_layer = tf.keras.Input(shape=(967, 32, 1))
    input_layer_aux = tf.keras.Input(shape=[1])

    # stack the layers
    convlayer = tf.keras.layers.Conv2D(128, (10,2))(input_layer)
    convlayer2 = tf.keras.layers.Conv2D(128, (10,2))(convlayer)
    flatten_layer = tf.keras.layers.Flatten()(convlayer2)
    flatten_layer_aux = tf.keras.layers.Flatten()(input_layer_aux)
    first_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(flatten_layer)
    second_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(first_dense)
    third_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(second_dense)
    concate = tf.keras.layers.Concatenate()([third_dense, input_layer_aux])
    output_layer = tf.keras.layers.Dense(32)(concate)

    # declare inputs and outputs
    func_model = tf.keras.models.Model(inputs=[input_layer, input_layer_aux], outputs=output_layer)

    return func_model

In [ ]:
#@title PREDICT
def predict(music, emotion, model, new_music):
    output_of_ONE_SONG = model.predict([music[0:1], emotion[0:1]])
    intSong = list(np.abs(np.round(output_of_ONE_SONG[0]/100, )).astype(int))

    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(26)]).reshape(-1))
    x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x))

    print(x_int_to_note)
    print(intSong)

    length = len(x_int_to_note.keys())
    print("length: " + str(length))

    predicted_notes = [x_int_to_note[i] for i in intSong]

    print("predicted notes")
    print(predicted_notes)
    print(len(predicted_notes))

    predicted_notes = list(map(float, predicted_notes))

    predicted_notes = np.round(predicted_notes / np.max(predicted_notes) * length).astype(int)

    return predicted_notes

In [ ]:
#@title CONVERT TO MIDI
def convert_to_midi(prediction_output):

    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:

        # pattern is a chord
        if type(pattern) == int:
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:

                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)


            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)

        # pattern is a note
        else:

            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [ ]:
#@title CLASSICAL CALM MODEL
def classicalCalm():

    musicPath ="/content/drive/MyDrive/classicalmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata'
    os.chdir(csvPath)

    annot = pd.read_csv('classical.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, calm_data, new_music = createClassicalCalmData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, calm_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/classical_calm.h5')

    predicted_notes = predict(bigX, calm_data, model, new_music)

    print (predicted_notes)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title CLASSICAL POWER MODEL
def classicalPower():

    musicPath ="/content/drive/MyDrive/classicalmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata/classical.csv'
    os.chdir(csvPath)

    annot = pd.read_csv('emotifyannotationsdata.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, power_data, new_music = createClassicalPowerData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, power_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/classical_power.h5')

    predicted_notes = predict(bigX, power_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title ROCK CALM MODEL
def rockCalm():

    musicPath ="/content/drive/MyDrive/rockmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata'
    os.chdir(csvPath)

    annot = pd.read_csv('rock.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, calm_data, new_music = createRockCalmData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, calm_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/rock_calm.h5')

    predicted_notes = predict(bigX, calm_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title ROCK POWER MODEL
def rockPower():

    musicPath ="/content/drive/MyDrive/rockmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata/rock.csv'
    os.chdir(csvPath)

    annot = pd.read_csv('emotifyannotationsdata.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, power_data, new_music = createRockPowerData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, power_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/rock_power.h5')

    predicted_notes = predict(bigX, power_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title ELECTRONIC CALM MODEL
def electronicCalm():

    musicPath ="/content/drive/MyDrive/electronicmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata/electronic.csv'
    os.chdir(csvPath)

    annot = pd.read_csv('emotifyannotationsdata.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, calm_data, new_music = createElectronicCalmData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, calm_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/electronic_calm.h5')

    predicted_notes = predict(bigX, calm_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title ELECTRONIC POWER MODEL
def electronicPower():

    musicPath ="/content/drive/MyDrive/electronicmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata/electronic.csv'
    os.chdir(csvPath)

    annot = pd.read_csv('emotifyannotationsdata.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    #CSV DATA

    print ("create final data")

    bigX, bigY, power_data, new_music = createElectronicPowerData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, power_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/electronic_power.h5')

    predicted_notes = predict(bigX, power_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title POP CALM MODEL
def popCalm():

    musicPath ="/content/drive/MyDrive/popmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata/pop.csv'
    os.chdir(csvPath)

    annot = pd.read_csv('emotifyannotationsdata.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, calm_data, new_music = createPopCalmData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, calm_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/pop_calm.h5')

    predicted_notes = predict(bigX, calm_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title POP POWER MODEL
def popPower():

    musicPath ="/content/drive/MyDrive/popmidi/"

    csvPath = '/content/drive/MyDrive/emotifyannotationsdata/pop.csv'
    os.chdir(csvPath)

    annot = pd.read_csv('emotifyannotationsdata.csv')
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]

    print ("create final data")

    bigX, bigY, power_data, new_music = createPopPowerData(musicPath, annot_subset)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae')

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, power_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=2,
            callbacks=[callback])

    model.save('/content/drive/MyDrive/emotifymusicMD/pop_power.h5')

    predicted_notes = predict(bigX, power_data, model, new_music)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    return Audio('output.wav')

In [ ]:
#@title MUSIC GENERATION
def music_generation():

    #PREDICT
    if Genre == 'classical' and Goal == 'relax':
        return classicalCalm() #26
    elif Genre == 'classical' and Goal == 'energize':
        return classicalPower() #4
    elif Genre == 'rock' and Goal == 'relax':
        return rockCalm() #12
    elif Genre == 'rock' and Goal == 'energize':
        return rockPower() #6
    elif Genre == 'electronic' and Goal == 'relax':
        return electronicCalm() #22
    elif Genre == 'electronic' and Goal == 'energize':
        return electronicPower() #9
    elif Genre == 'pop' and Goal == 'relax':
        return popCalm() #19
    elif Genre == 'pop' and Goal == 'energize':
        return popPower() #4
    else:
        print("invalid input")

# App

In [ ]:
Genre = 'classical' #@param ["classical", "rock", "electronic", "pop"]
Goal = 'relax' #@param ["relax", "energize"]

In [ ]:
music_generation()